In [2]:
# skin_tone_detection_app.py
import gradio as gr
import cv2
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import tensorflow as tf
import pickle
import os

print("🚀 Starting Skin Tone Detection Application...")

# =============================================================================
# 1. LOAD PRE-TRAINED MODEL AND DATA
# =============================================================================

def load_model_and_data():
    """Load the pre-trained model and skin tone categories"""
    try:
        # Load the trained model
        print("📦 Loading pre-trained model...")
        model = tf.keras.models.load_model("/content/skin_tone_model.h5")
        print("✅ Model loaded successfully!")

        # Load skin tone categories from PKL file
        print("📦 Loading skin tone categories...")
        with open('/content/skin_tone_categories.pkl', 'rb') as f:
            skin_tone_categories = pickle.load(f)
        print("✅ Skin tone categories loaded successfully!")

        return model, skin_tone_categories

    except Exception as e:
        print(f"❌ Error loading files: {e}")
        print("Please make sure 'skin_tone_model.h5' and 'skin_tone_categories.pkl' are in the current directory")
        return None, None

# Load model and categories
model, skin_tone_categories = load_model_and_data()

# =============================================================================
# 2. CORE PREDICTION FUNCTIONS (No Training Needed)
# =============================================================================

def predict_skin_tone(image):
    """
    Predict skin tone from an input image using pre-trained model
    """
    # Convert image to RGB if it's in BGR
    if len(image.shape) == 3 and image.shape[2] == 3:
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    else:
        image_rgb = image

    # Detect face
    gray = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2GRAY)
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    faces = face_cascade.detectMultiScale(gray, 1.1, 5)

    if len(faces) == 0:
        return "❌ No face detected. Please upload a clear face photo.", None, None

    # Get the first face
    x, y, w, h = faces[0]

    # Add padding around the face
    padding = 20
    x = max(0, x - padding)
    y = max(0, y - padding)
    w = min(image.shape[1] - x, w + 2 * padding)
    h = min(image.shape[0] - y, h + 2 * padding)

    face_img = image_rgb[y:y+h, x:x+w]

    # Preprocess the face image
    face_resized = cv2.resize(face_img, (128, 128))
    face_normalized = face_resized.astype('float32') / 255.0
    face_expanded = np.expand_dims(face_normalized, axis=0)

    # Make prediction using pre-trained model
    predictions = model.predict(face_expanded, verbose=0)
    predicted_class = np.argmax(predictions, axis=1)[0]
    confidence = np.max(predictions)

    # Get color information from loaded categories
    color_info = skin_tone_categories[predicted_class]

    # Create result text
    result_text = f"""
🎯 Predicted Skin Tone: {color_info['name']}
📊 Confidence: {confidence:.2%}
🎨 Hex Code: {color_info['hex']}
🌈 RGB Values: {color_info['rgb']}
"""

    return result_text, face_img, color_info

def get_color_recommendations(skin_tone_name):
    """
    Rule-based color recommendations for different skin tones
    """
    recommendations = {
        "Ivory": {
            "best": ["Navy Blue", "Emerald Green", "Burgundy", "Soft Pink", "Lavender"],
            "good": ["Pastel Blue", "Cream", "Dusty Rose", "Mint Green", "Light Gray"],
            "avoid": ["Neon Colors", "Bright Yellow", "Orange", "Pure White"]
        },
        "Beige": {
            "best": ["Olive Green", "Terracotta", "Mustard Yellow", "Warm Brown", "Coral"],
            "good": ["Peach", "Gold", "Cream", "Soft Red", "Tan"],
            "avoid": ["Electric Blue", "Hot Pink", "Black", "Bright Green"]
        },
        "Sand": {
            "best": ["Turquoise", "Coral", "Warm Red", "Gold", "Earth Tones"],
            "good": ["Orange", "Yellow", "Brown", "Cream", "Warm Gray"],
            "avoid": ["Pastel Colors", "Cool Blues", "Lavender", "Silver"]
        },
        "Honey": {
            "best": ["Royal Blue", "Purple", "Red", "Green", "Gold"],
            "good": ["Orange", "Yellow", "Brown", "Warm Colors", "Cream"],
            "avoid": ["Pale Colors", "Cool Pastels", "Gray", "Silver"]
        },
        "Caramel": {
            "best": ["Bright White", "Royal Blue", "Fuchsia", "Emerald Green", "Red"],
            "good": ["Purple", "Gold", "Orange", "Yellow", "Warm Colors"],
            "avoid": ["Brown", "Black", "Dark Colors", "Muted Tones"]
        },
        "Espresso": {
            "best": ["Vibrant Colors", "Bright White", "Electric Blue", "Hot Pink", "Red"],
            "good": ["Purple", "Green", "Yellow", "Orange", "Metallics"],
            "avoid": ["Dark Brown", "Black", "Navy Blue", "Gray"]
        }
    }

    return recommendations.get(skin_tone_name, {
        "best": ["Neutral Colors", "Classic Shades"],
        "good": ["Most Colors", "Experiment Freely"],
        "avoid": ["Extreme Contrasts"]
    })

# =============================================================================
# 3. COLOR VISUALIZATION FUNCTIONS
# =============================================================================

def get_color_hex(color_name):
    """
    Map color names to hex codes
    """
    color_map = {
        "Navy Blue": "#000080", "Emerald Green": "#50C878", "Burgundy": "#800020",
        "Soft Pink": "#FFB6C1", "Lavender": "#E6E6FA", "Pastel Blue": "#AEC6CF",
        "Cream": "#FFFDD0", "Dusty Rose": "#DCB4BC", "Mint Green": "#98FB98",
        "Light Gray": "#D3D3D3", "Olive Green": "#808000", "Terracotta": "#E2725B",
        "Mustard Yellow": "#FFDB58", "Warm Brown": "#964B00", "Coral": "#FF7F50",
        "Peach": "#FFE5B4", "Gold": "#FFD700", "Soft Red": "#FA8072", "Tan": "#D2B48C",
        "Turquoise": "#40E0D0", "Warm Red": "#FF5349", "Earth Tones": "#A0522D",
        "Orange": "#FFA500", "Yellow": "#FFFF00", "Warm Gray": "#808080",
        "Royal Blue": "#4169E1", "Purple": "#800080", "Red": "#FF0000",
        "Green": "#008000", "Bright White": "#FFFFFF", "Fuchsia": "#FF00FF",
        "Electric Blue": "#7DF9FF", "Hot Pink": "#FF69B4", "Vibrant Colors": "#FF0000",
        "Metallics": "#D4AF37", "Dark Brown": "#654321", "Black": "#000000",
        "Neon Colors": "#39FF14", "Bright Yellow": "#FFFF00", "Pure White": "#FFFFFF",
        "Bright Green": "#00FF00", "Pastel Colors": "#FFB6C1", "Cool Blues": "#ADD8E6",
        "Silver": "#C0C0C0", "Pale Colors": "#FADADD", "Cool Pastels": "#E0FFFF",
        "Gray": "#808080", "Dark Colors": "#2F4F4F", "Muted Tones": "#708090"
    }
    return color_map.get(color_name, "#CCCCCC")

def create_visual_color_recommendations(color_info, recommendations):
    """
    Create visual color swatches for recommendations
    """
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))

    # Main skin tone
    axes[0, 0].add_patch(Rectangle((0, 0), 1, 1, color=np.array(color_info['rgb'])/255))
    axes[0, 0].set_title(f"🎯 Your Skin Tone: {color_info['name']}", fontsize=16, fontweight='bold', pad=20)
    axes[0, 0].text(0.5, -0.1, f"Hex: {color_info['hex']}\nRGB: {color_info['rgb']}",
                   ha='center', transform=axes[0, 0].transAxes, fontsize=12)
    axes[0, 0].axis('off')

    # Best colors
    best_colors = recommendations['best']
    axes[0, 1].set_title("⭐ BEST Colors for You", fontsize=16, fontweight='bold', pad=20)
    for i, color_name in enumerate(best_colors):
        color_hex = get_color_hex(color_name)
        if color_hex:
            axes[0, 1].add_patch(Rectangle((i*0.2, 0.6), 0.18, 0.3, color=color_hex, edgecolor='black', linewidth=1))
            axes[0, 1].text(i*0.2 + 0.09, 0.4, color_name, ha='center', va='top', fontsize=9, wrap=True)
            axes[0, 1].text(i*0.2 + 0.09, 0.3, color_hex, ha='center', va='top', fontsize=8, color='gray')
    axes[0, 1].set_xlim(0, 1)
    axes[0, 1].set_ylim(0, 1)
    axes[0, 1].axis('off')

    # Good colors
    good_colors = recommendations['good']
    axes[1, 0].set_title("👍 GOOD Colors", fontsize=16, fontweight='bold', pad=20)
    for i, color_name in enumerate(good_colors):
        color_hex = get_color_hex(color_name)
        if color_hex:
            axes[1, 0].add_patch(Rectangle((i*0.2, 0.6), 0.18, 0.3, color=color_hex, edgecolor='black', linewidth=1))
            axes[1, 0].text(i*0.2 + 0.09, 0.4, color_name, ha='center', va='top', fontsize=9, wrap=True)
            axes[1, 0].text(i*0.2 + 0.09, 0.3, color_hex, ha='center', va='top', fontsize=8, color='gray')
    axes[1, 0].set_xlim(0, 1)
    axes[1, 0].set_ylim(0, 1)
    axes[1, 0].axis('off')

    # Avoid colors
    avoid_colors = recommendations['avoid']
    axes[1, 1].set_title("🚫 AVOID These Colors", fontsize=16, fontweight='bold', pad=20)
    for i, color_name in enumerate(avoid_colors):
        color_hex = get_color_hex(color_name)
        if color_hex:
            axes[1, 1].add_patch(Rectangle((i*0.2, 0.6), 0.18, 0.3, color=color_hex, edgecolor='black', linewidth=1))
            axes[1, 1].text(i*0.2 + 0.09, 0.4, color_name, ha='center', va='top', fontsize=9, wrap=True)
            axes[1, 1].text(i*0.2 + 0.09, 0.3, color_hex, ha='center', va='top', fontsize=8, color='gray')
    axes[1, 1].set_xlim(0, 1)
    axes[1, 1].set_ylim(0, 1)
    axes[1, 1].axis('off')

    plt.tight_layout()
    return fig

# =============================================================================
# 4. GRADIO UI SETUP
# =============================================================================

def create_skin_tone_app():
    """
    Create the main Gradio application
    """

    def process_image(input_image):
        """
        Main processing function for the UI
        """
        try:
            if input_image is None:
                return "❌ Please upload an image", None, None, "Waiting for image..."

            # Convert to OpenCV format
            image_np = np.array(input_image)
            image_bgr = cv2.cvtColor(image_np, cv2.COLOR_RGB2BGR)

            # Get prediction
            result_text, face_img, color_info = predict_skin_tone(image_bgr)

            if "No face detected" in result_text:
                # Create placeholder plot
                fig, ax = plt.subplots(figsize=(10, 6))
                ax.text(0.5, 0.5, "👤 No Face Detected\n\nPlease upload a clear face photo",
                       ha='center', va='center', fontsize=16, transform=ax.transAxes)
                ax.set_facecolor('#f0f0f0')
                ax.axis('off')
                return result_text, None, fig, "No face detected - try another photo"

            # Get recommendations
            skin_tone_name = color_info['name']
            recommendations = get_color_recommendations(skin_tone_name)

            # Create visualizations
            fig = create_visual_color_recommendations(color_info, recommendations)

            # Create text recommendations
            rec_text = f"🎨 COLOR RECOMMENDATIONS FOR {skin_tone_name.upper()}:\n\n"
            rec_text += "⭐ BEST Colors:\n" + "\n".join([f"   • {color}" for color in recommendations['best']]) + "\n\n"
            rec_text += "👍 GOOD Colors:\n" + "\n".join([f"   • {color}" for color in recommendations['good']]) + "\n\n"
            rec_text += "🚫 AVOID Colors:\n" + "\n".join([f"   • {color}" for color in recommendations['avoid']])

            # Convert face image for display
            if face_img is not None:
                face_img_rgb = cv2.cvtColor(face_img, cv2.COLOR_BGR2RGB)
                return result_text, face_img_rgb, fig, rec_text
            else:
                return result_text, None, fig, rec_text

        except Exception as e:
            error_msg = f"❌ Error: {str(e)}"
            print(error_msg)
            # Create error plot
            fig, ax = plt.subplots(figsize=(10, 6))
            ax.text(0.5, 0.5, "❌ Processing Error\n\nPlease try another image",
                   ha='center', va='center', fontsize=16, transform=ax.transAxes)
            ax.set_facecolor('#ffe6e6')
            ax.axis('off')
            return error_msg, None, fig, "Error occurred during processing"

    # Create the Gradio interface
    with gr.Blocks(theme=gr.themes.Soft(), title="Skin Tone Detection App") as demo:
        gr.Markdown(
            """
            # 🎨 Skin Tone Detection & Color Analysis
            **Upload a face photo to discover your skin tone and get personalized color recommendations!**

            *Powered by Deep Learning - No retraining needed!*
            """
        )

        with gr.Row():
            with gr.Column(scale=1):
                gr.Markdown("### 📸 Input")
                input_image = gr.Image(
                    label="Upload Face Photo",
                    type="pil",
                    height=300
                )
                analyze_btn = gr.Button(
                    "🎯 Analyze Skin Tone",
                    variant="primary",
                    size="lg"
                )

                # Model info
                with gr.Accordion("📊 Model Information", open=False):
                    gr.Markdown(f"""
                    **Pre-trained Model Status:** ✅ Loaded Successfully
                    **Skin Tone Categories:** ✅ {len(skin_tone_categories)} categories loaded
                    **Input Size:** 128x128 pixels
                    **Model Architecture:** Custom CNN
                    """)

            with gr.Column(scale=2):
                gr.Markdown("### 📊 Results")
                with gr.Row():
                    result_output = gr.Textbox(
                        label="Skin Tone Analysis",
                        lines=4,
                        max_lines=6
                    )
                    face_output = gr.Image(
                        label="Detected Face Region",
                        height=200,
                        interactive=False
                    )

                gr.Markdown("### 🎨 Color Recommendations")
                plot_output = gr.Plot(
                    label="Visual Color Guide"
                )

                detailed_output = gr.Textbox(
                    label="Detailed Recommendations",
                    lines=8,
                    max_lines=12
                )

        # Instructions
        with gr.Accordion("📖 How to Use", open=True):
            gr.Markdown("""
            1. **Upload** a clear face photo using the upload button
            2. **Click** the "Analyze Skin Tone" button
            3. **View** your skin tone analysis and color recommendations
            4. **Use** the recommendations for clothing, makeup, and styling choices

            **Tips for best results:**
            - Use good natural lighting
            - Ensure face is clearly visible
            - Avoid heavy makeup or filters
            - Use neutral background
            """)

        # Connect the button
        analyze_btn.click(
            fn=process_image,
            inputs=[input_image],
            outputs=[result_output, face_output, plot_output, detailed_output]
        )

    return demo

# =============================================================================
# 5. LAUNCH APPLICATION
# =============================================================================

def main():
    """
    Main function to launch the application
    """
    if model is None or skin_tone_categories is None:
        print("❌ Cannot start application - required files not loaded")
        return

    print("🎯 Starting Skin Tone Detection Application...")
    print("📊 Model Information:")
    print(f"   - Skin tone categories: {len(skin_tone_categories)}")
    print(f"   - Model input shape: {model.input_shape}")
    print(f"   - Model output classes: {model.output_shape[1]}")

    # Create and launch the app
    app = create_skin_tone_app()

    print("🚀 Launching Web Interface...")
    print("🌐 The app will open in your browser")
    print("🔗 A public share link will be created")
    print("⏰ This may take a few moments...")

    # Launch with sharing enabled
    app.launch(
        share=True,
        inbrowser=True,
        debug=False,
        show_error=True
    )

# Run the application
if __name__ == "__main__":
    main()

🚀 Starting Skin Tone Detection Application...
📦 Loading pre-trained model...


✅ Model loaded successfully!
📦 Loading skin tone categories...
✅ Skin tone categories loaded successfully!
🎯 Starting Skin Tone Detection Application...
📊 Model Information:
   - Skin tone categories: 6
   - Model input shape: (None, 128, 128, 3)
   - Model output classes: 6
🚀 Launching Web Interface...
🌐 The app will open in your browser
🔗 A public share link will be created
⏰ This may take a few moments...
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://7aa2443a71bc9d87ec.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
